### SRT Example

In [39]:
import pandas as pd
pd.set_option('display.max_rows', None)

#### Deal Structure

* waterfall
  * Distribution Day : 
    * Principal : `Pro Rata` to senior (A) and protected tranche (B)
    * Interest : "issuer" will pay interest to tranches
  * End of Colleciton : allocate pool loss by move fund from collateral cash account to "Issuer"
  * Closing Day: fund collateral cash account with SRT tranche balance
  * Clean up: when deal was called, pay all cash from collateral cash account to protected tranche (B)
  

In [40]:
from absbox import Generic

seniorBalance = 1000
srtTrancheBal = 200
srtRate = 0.08
closingDate = "2021-06-15"
periodPrincipal = ("curPoolCollection", None, "Principal","Prepayment","Recovery")
reinvestRate = 0.00


srt01 = Generic(
    "SRT Example"
    ,{"cutoff":"2021-06-01","closing":closingDate,"firstPay":"2021-07-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":24
          ,"freq":"Monthly","type":"Level","originDate":"2021-06-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":24
          ,"status":"current"}]],
     }
    ,(("acc01",{"balance":0})
      ,("srtAcc",{"balance":0.0
                  ,"interest":{"period":"QuarterEnd"
                               ,"rate":reinvestRate
                               ,"lastSettleDate":closingDate}})
      ,("dummy",{"balance":0.0})
      ,)
    ,(("A1",{"balance":seniorBalance
             ,"rate":0.00
             ,"originBalance":seniorBalance
             ,"originRate":0.00
             ,"startDate":closingDate
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":0.0
             ,"rate":srtRate
             ,"originBalance":0
             ,"originRate":srtRate
             ,"startDate":closingDate
             ,"rateType":{"Fixed":srtRate}
             ,"bondType":{"Sequential":None}
             }))
    ,tuple()
    ,{"amortizing":[
          # pay prorata to senior and SRT tranch
         ['accrueAndPayInt',"dummy",["A1"],{"support":["facility","originator"]}]
         ,["calcInt","B"]
         ,["payInt","dummy",["B"],{"support":["facility","originator"]}]

         ,["calcBondPrin",["A1","B"],{"formula":periodPrincipal}] 

         ,["payPrinWithDue","acc01",["A1"]]         
         ,["payPrinWithDue","srtAcc",["B"]]

     ],
      "closingDay":[
                    ["fundWith","srtAcc","B",{"formula":("const",srtTrancheBal)}]
                   ],
      "endOfCollection":[
          # draw loss amount and pay to originator
          ["liqRepayResidual","srtAcc", "originator", {"formula":("curPoolCollection",None,"Losses")}]
      ],
      "cleanUp":[
          ["payIntResidual","srtAcc","B"]
      ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,{"originator":{"type" : "Unlimited", "start": closingDate}}
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [41]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english', check=False)

r = localAPI.run(srt01
                ,poolAssump=("Pool",("Mortgage", {"CDR":0.01}, None, None, None)
                                   ,None
                                   ,None)
                ,runAssump=[("call",("poolFactor",0.10))]
                ,read=True)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.4, server:0.27.17

Warning Message from server:
Bond B is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off

#### View Bond Cashflow

In [42]:
from absbox import readBondsCf

readBondsCf(r['bonds']).head()

Bond             A1                                      B                     \
Field       balance interest principal rate   cash balance interest principal   
date                                                                            
2021-06-15      NaN      NaN       NaN  NaN    NaN  200.00     0.00   -200.00   
2021-07-20  1000.00      0.0      0.00  0.0   0.00  200.00     1.53      0.00   
2021-08-20   929.37      0.0     70.63  0.0  70.63  185.88     1.35     14.12   
2021-09-20   858.33      0.0     71.04  0.0  71.04  171.67     1.26     14.21   
2021-10-20   786.88      0.0     71.45  0.0  71.45  157.38     1.12     14.29   

Bond                     
Field       rate   cash  
date                     
2021-06-15  0.00   0.00  
2021-07-20  0.08   1.53  
2021-08-20  0.08  15.47  
2021-09-20  0.08  15.47  
2021-10-20  0.08  15.41

#### Pool Loss metrics

Here, we can identify total pool loss is `23.11`

In [43]:
r['pool']['flow'].Loss.sum()

23.11

How much pool loss was cured via SRT traction ? we can filter out transaction in the `srtAcc` ,the collateral cash account draw out 

In [44]:
r['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].head()

,balance,change,memo
date,,,
2021-06-15,200.00,0.00,<Support:originator>
2021-06-30,200.00,0.00,<Support:originator>
2021-07-31,198.19,-1.81,<Support:originator>
2021-08-31,182.27,-1.80,<Support:originator>
2021-09-30,166.34,-1.72,<Support:originator>


Summing up all loss cured by SRT account, that's total cured `18.11`

In [45]:
r['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].change.sum()

-18.11

We can tie out `18.11` loss in the SRT tranche (B). There 18.11 oustanding balance at end of projection.

In [46]:
r['bonds']['B'].tail()

,balance,interest,principal,rate,cash,factor,memo
date,,,,,,,
2023-01-20,18.11,0.12,0.0,0.08,0.12,None,"[<PayInt:B>, <PayPrin:B>]"
2023-02-20,18.11,0.12,0.0,0.08,0.12,None,"[<PayInt:B>, <PayPrin:B>]"
2023-03-20,18.11,0.11,0.0,0.08,0.11,None,"[<PayInt:B>, <PayPrin:B>]"
2023-04-20,18.11,0.12,0.0,0.08,0.12,None,"[<PayInt:B>, <PayPrin:B>]"
2023-05-20,18.11,0.11,0.0,0.08,0.11,None,"[[<PayInt:B>, <PayPrin:B>], <PayYield:B>]"


#### Return of SRT transaction

Let's calculate the IRR of protect tranche

In [47]:
r['bonds']['B'].head()

,balance,interest,principal,rate,cash,factor,memo
date,,,,,,,
2021-06-15,200.00,0.00,-200.00,0.00,0.00,None,"<FundWith:B,200.00>"
2021-07-20,200.00,1.53,0.00,0.08,1.53,None,"[<PayInt:B>, <PayPrin:B>]"
2021-08-20,185.88,1.35,14.12,0.08,15.47,None,"[<PayInt:B>, <PayPrin:B>]"
2021-09-20,171.67,1.26,14.21,0.08,15.47,None,"[<PayInt:B>, <PayPrin:B>]"
2021-10-20,157.38,1.12,14.29,0.08,15.41,None,"[<PayInt:B>, <PayPrin:B>]"


Let's build a simple irr function

In [48]:
from pyxirr import xirr

def calcIRR(df, init):
    investDate,investAmount = init
    ds = [investDate] + df.index.to_list()
    vs = [investAmount] + df.cash.to_list()
    return xirr(ds, vs)

calcIRR(r['bonds']['B'], ("2021-06-15",-200))

-0.04323574436198406

##### Sensitiviy Analysis : Pool Perf vs. IRR

let's assump how different prepayment behavior would impact on the IRR

1. build an assumption Map
2. run with ..runByScenarios()


In [49]:
from lenses import lens

myAssumption = ("Pool",("Mortgage",None,None,None,None)
                                ,None
                                ,None)
myAssumption2 = myAssumption & lens[1][2].set({"CPR":0.01})
myAssumption3 = myAssumption & lens[1][2].set({"CPR":0.02})


rs = localAPI.runByScenarios(srt01
                            ,poolAssump={"CPR-0":myAssumption
                                        ,"CPR-1":myAssumption2
                                        ,"CPR-2":myAssumption3
                                       }
                            ,read=True)

Warning Message from server:
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off

View pool cashflow from multiple scenarios

In [50]:
from absbox import readMultiFlowsByScenarios
readMultiFlowsByScenarios(rs, (lens['pool']['flow'],["Balance",'Prepayment'])).head()

Scenario      CPR-0               CPR-1               CPR-2           
Field       Balance Prepayment  Balance Prepayment  Balance Prepayment
Date                                                                  
2021-06-15  2200.00          0  2200.00       0.00  2200.00       0.00
2021-06-30  2200.00          0  2200.00       0.00  2200.00       0.00
2021-07-31  2115.17          0  2113.43       1.81  2111.66       3.65
2021-08-31  2029.77          0  2026.37       1.80  2022.93       3.62
2021-09-30  1943.81          0  1938.90       1.72  1933.94       3.46

In [51]:
import toolz as tz

tz.valmap(lambda x: calcIRR(x['bonds']['B'], ("2021-06-15",-200)), rs)

{'CPR-0': 0.0822966584499679,
 'CPR-1': 0.08250918429431883,
 'CPR-2': 0.08235205608296689}

Whoa, interesting ! CPR=1% will yield most IRR

#### What's the protection exposure ? 

To what extend the current capital structure will hedge the default risk ? 


let's assump how different default stress would impact on the IRR

1. build an assumption Map
2. run with ..runByScenarios()

##### Sensitiviy Analysis : Pool Perf vs. Exposed Loss

Exposed Loss : the loss not being hedged by SRT tranche

In [52]:
myAssumption = ("Pool",("Mortgage",None,None,None,None)
                                ,None
                                ,None)
myAssumption2 = myAssumption & lens[1][1].set({"CDR":0.01})
myAssumption3 = myAssumption & lens[1][1].set({"CDR":0.02})


rs = localAPI.runByScenarios(srt01
                            ,poolAssump={"CDR-0":myAssumption
                                        ,"CDR-1":myAssumption2
                                        ,"CDR-2":myAssumption3
                                       }
                            ,read=True)

Warning Message from server:
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off
Bond B is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off
Bond B is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off

View pool cashflows

In [53]:
from absbox import readMultiFlowsByScenarios
readMultiFlowsByScenarios(rs, (lens['pool']['flow'],["Balance",'Default'])).head()

Scenario      CDR-0            CDR-1            CDR-2        
Field       Balance Default  Balance Default  Balance Default
Date                                                         
2021-06-15  2200.00       0  2200.00    0.00  2200.00    0.00
2021-06-30  2200.00       0  2200.00    0.00  2200.00    0.00
2021-07-31  2115.17       0  2113.43    1.81  2111.66    3.65
2021-08-31  2029.77       0  2026.37    1.80  2022.93    3.62
2021-09-30  1943.81       0  1938.90    1.72  1933.94    3.46

By writing a small function to calculate unhendge amount,we are able to tell in all the scenarios, the unhedge amount

In [54]:
from pyxirr import xirr

def unHedgeAmount(x:dict):
    "x is the single run result"
    
    poolLoss = x['pool']['flow'].Loss.sum()
    hedgedAmount = x['accounts']['srtAcc'].loc[lambda df: df.memo == "<Support:originator>"].change.sum()
    
    return max(poolLoss - hedgedAmount,0)

tz.valmap(unHedgeAmount, rs)

{'CDR-0': 0.0, 'CDR-1': 41.449999999999996, 'CDR-2': 81.19999999999999}

#### Others ?

Actuall `absbox` is flexible enough to perform sensitivity analysis on any two variables :

* It could be reinvestment rate v.s IRR on SRT tranche..
* It could be capital structure v.s IRR on SRT tranche..

### SRT with 3 tranches

The key for 3 tranches is , the `srtAcc` only move funds to the loss which is the excess amount over the first loss tranche

In [55]:
firstLossTranche = 5

formulaToCurLoss = ("excess", ("cumPoolNetLoss",)
                            , ("abs",("accountTxnAmount","<Support:originator>","srtAcc"))
                            , ("originalBondBalance","C"))

In [56]:
seniorBalance = 1000
srtTrancheBal = 200

srtRate = 0.08
closingDate = "2021-06-15"
periodPrincipal = ("curPoolCollection", None, "Principal","Prepayment","Recovery")
reinvestRate = 0.00



srt02 = Generic(
    "SRT Example"
    ,{"cutoff":"2021-06-01","closing":closingDate,"firstPay":"2021-07-20"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":24
          ,"freq":"Monthly","type":"Level","originDate":"2021-06-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":24
          ,"status":"current"}]],
     }
    ,(("acc01",{"balance":0})
      ,("srtAcc",{"balance":0.0
                  ,"interest":{"period":"QuarterEnd"
                               ,"rate":reinvestRate
                               ,"lastSettleDate":closingDate}})
      ,("dummy",{"balance":0.0})
      ,)
    ,(("A1",{"balance":seniorBalance
             ,"rate":0.00
             ,"originBalance":seniorBalance
             ,"originRate":0.00
             ,"startDate":closingDate
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":0.0
             ,"rate":srtRate
             ,"originBalance":0
             ,"originRate":srtRate
             ,"startDate":closingDate
             ,"rateType":{"Fixed":srtRate}
             ,"bondType":{"Sequential":None}
             })
      ,("C",{"balance":firstLossTranche
             ,"rate":0.0
             ,"originBalance":firstLossTranche
             ,"originRate":0.0
             ,"startDate":closingDate
             ,"rateType":{"Fixed":0.0}
             ,"bondType":{"Sequential":None}
             })     
     )
    ,tuple()
    ,{"amortizing":[
          # pay prorata to senior and SRT tranch
         ["calcInt","B"]
         ,["payInt","dummy",["B"],{"support":["facility","originator"]}]
         ,["calcBondPrin",["A1","B"],{"formula":periodPrincipal}] 
         ,["payPrinWithDue","acc01",["A1"]]         
         ,["payPrinWithDue","srtAcc",["B"]]
         ,["If", [('isPaidOff',"A1","B"),True]
           ,["payPrin","acc01",["C"]]]
     ],
      "closingDay":[["fundWith","srtAcc","B",{"formula":("const",srtTrancheBal)}]],
      "endOfCollection":[
          # draw loss amount and pay to originator
          ["liqRepayResidual","srtAcc", "originator", {"formula":formulaToCurLoss}]
      ],
      "cleanUp":[
          ["payIntResidual","srtAcc","B"]
      ]
     }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,{"originator":{"type" : "Unlimited", "start": closingDate}}
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [57]:
from absbox import API,EnginePath

r02 = localAPI.run(srt02
                ,poolAssump=("Pool",("Mortgage", {"CDR":0.01}, None, None, None)
                                   ,None
                                   ,None)
                ,runAssump=[("call",("poolFactor",0.10))
                           ,("inspect"
                             ,("MonthEnd",("excess", ("cumPoolNetLoss",)
                                                 , ("abs",("accountTxnAmount","<Support:originator>","srtAcc"))
                                                 , ("constant",firstLossTranche))
                                          ))
                            ]
                ,read=True)

Warning Message from server:
Bond B is not paid off
Bond C is not paid off
Account acc01 has cash to be distributed
LiquidityProvider originator is not paid off

In [58]:
readBondsCf(r02['bonds']).head()

Bond             A1                                      B                     \
Field       balance interest principal rate   cash balance interest principal   
date                                                                            
2021-06-15      NaN      NaN       NaN  NaN    NaN  200.00     0.00   -200.00   
2021-07-20  1000.00      0.0      0.00  0.0   0.00  200.00     1.53      0.00   
2021-08-20   929.37      0.0     70.63  0.0  70.63  185.88     1.35     14.12   
2021-09-20   858.33      0.0     71.04  0.0  71.04  171.67     1.26     14.21   
2021-10-20   786.88      0.0     71.45  0.0  71.45  157.38     1.12     14.29   

Bond                     
Field       rate   cash  
date                     
2021-06-15  0.00   0.00  
2021-07-20  0.08   1.53  
2021-08-20  0.08  15.47  
2021-09-20  0.08  15.47  
2021-10-20  0.08  15.41

Now , the loss from the pool only will be cured from `srtAcc` account if cumulative loss is greater than `5`

Here, at `2021-09-30`, the cumulative loss is 5.33 from the pool, the engine will only allocate `0.33` 

In [59]:
pd.concat([r02['accounts']['srtAcc'],r02['pool']['flow'].CumLoss.to_frame()]).sort_index().loc["2021-09-30"]

,balance,change,memo,CumLoss
2021-09-30,171.67,0.00,<BankInterest:>,NaN
2021-09-30,171.34,-0.33,<Support:originator>,NaN
2021-09-30,NaN,NaN,NaN,5.33


#### How thickness of FirstLossTranche would affect the IRR ?

In [60]:
firstLossTrancheDraft = {"FirstLoss-1": 5,"FirstLoss-2":10,"FirstLoss-3":20}

build deal structures with a map( and lenses !)

In [61]:
dealDrafts = tz.valmap(lambda x: srt02 & lens.bonds[2][1].Fork(lens['balance'],lens['originBalance']).set(x) 
                     ,firstLossTrancheDraft)

run differnt deal structures with funciton `runStructs()`

In [62]:
rm02 = localAPI.runStructs(dealDrafts
                        ,poolAssump=("Pool",("Mortgage", {"CDR":0.01}, None, None, None)
                                           ,None
                                           ,None)
                        ,nonPoolAssump=[("call",("poolFactor",0.10))
                                   ,("inspect"
                                     ,("MonthEnd",("excess", ("cumPoolNetLoss",)
                                                         , ("abs",("accountTxnAmount","<Support:originator>","srtAcc"))
                                                         , ("constant",firstLossTranche))
                                                  ))
                                    ]
                        ,read=True)

In [63]:
tz.valmap(lambda x: calcIRR(x['bonds']['B'], ("2021-06-15",-200)), rm02)

{'FirstLoss-1': -0.008036217135081734,
 'FirstLoss-2': 0.020824974743072763,
 'FirstLoss-3': 0.08232329854365725}